In [1]:
import numpy as np
import matplotlib.pyplot as plt

import pandas as pd

import scipy

In [2]:
import seaborn as sns

In [3]:
import sys
sys.path.append('..')

from Approximators.Bernstein import CauchySimplex

In [4]:
from utils import solve_multiple_coefficient_bessel

Consider Bessel's differential equation with initial conditions $y(0)=y(1)=0$. Let $J_m(x)$ and $Y_m(x)$ be Bessel functions of the first and second kind, respectively. Then, this differential equation has solutions of the form
\begin{align}
    y = c_1J_m(\sqrt{\lambda} x) + c_2Y_m(\sqrt{\lambda} x),
\end{align}
for constants $c_1, c_2$. To satisfy the initial conditions, we must have that $c_2=0$ and $J_m(\sqrt{\lambda})=0$. Thus, the eigenvalues of Bessel's differential equation are the square of the roots of the $m$-th Bessel's function, $J_m$.

Take the parameterization $x=e^z-1$. Then this differential equation can be rewritten as
\begin{align}
    (1-e^{-z})^2y''(z)\ +\ e^{-z}(1-e^{-z})y'(z)\ +\ (\lambda(e^z-1)^2-m^2)y(z)\ =\ 0,
\end{align}
with initial conditions $y(0)=y(\ln(2))=0$.

Take the parameterization $x=e^{az}-1$. Then this differential equation can be rewritten as
\begin{align}
    \frac{1}{a^2}(1-e^{-az})^2y''(z)\ +\ \frac{1}{a}e^{-az}(1-e^{-az})y'(z)\ +\ (\lambda(e^{az}-1)^2-m^2)y(z)\ =\ 0,
\end{align}
with initial conditions $y(0)=y(\ln(2)/a)=0$.


In [5]:
m = 2
a = 1

In [6]:
x = np.linspace(0, np.log(2) / a, 256)
y = [(1 - np.exp(-a * x)) ** 2 / (a ** 2), np.exp(-a * x) * (1 - np.exp(-a * x)) / a, 
     (np.exp(a * x) - 1) ** 2]

In [7]:
n_eigenvals = 20

In [8]:
n_vals = np.arange(4, 15 + 1, 1)

In [9]:
true_eigenvalues = scipy.special.jn_zeros(m, n_eigenvals)

# Polynomial Approximation

In [10]:
polynomial_results = []

In [11]:
for n in n_vals:
    print(f"Starting n = {n}")

    polynomial_approximator = CauchySimplex(n, 0).fit(x, y)
    
    solver = solve_multiple_coefficient_bessel(polynomial_approximator, a, m, Lx=0, Ux=np.log(2)/a, 
                                               Nx=256, dtype=np.float64, n_eigenvals=n_eigenvals)
    
    evals = np.sort(solver.eigenvalues.real)

    ratio_dataframe = pd.DataFrame(abs(np.sqrt(evals) - true_eigenvalues), columns=['Eigenvalue Errors'])
    ratio_dataframe['Approximator'] = 'Polynomial'
    ratio_dataframe['Num. Coefs'] = n
    
    polynomial_results.append(ratio_dataframe)

Starting n = 4
2023-10-02 12:17:45,936 subsystems 0/1 INFO :: Building subproblem matrices 1/1 (~100%) Elapsed: 0s, Remaining: 0s, Rate: 4.7e+00/s
Starting n = 5
2023-10-02 12:17:46,208 subsystems 0/1 INFO :: Building subproblem matrices 1/1 (~100%) Elapsed: 0s, Remaining: 0s, Rate: 4.0e+00/s
Starting n = 6
2023-10-02 12:17:46,519 subsystems 0/1 INFO :: Building subproblem matrices 1/1 (~100%) Elapsed: 0s, Remaining: 0s, Rate: 3.5e+00/s
Starting n = 7
2023-10-02 12:17:46,882 subsystems 0/1 INFO :: Building subproblem matrices 1/1 (~100%) Elapsed: 0s, Remaining: 0s, Rate: 3.0e+00/s
Starting n = 8
2023-10-02 12:17:47,273 subsystems 0/1 INFO :: Building subproblem matrices 1/1 (~100%) Elapsed: 0s, Remaining: 0s, Rate: 2.7e+00/s
Starting n = 9
2023-10-02 12:17:47,701 subsystems 0/1 INFO :: Building subproblem matrices 1/1 (~100%) Elapsed: 0s, Remaining: 0s, Rate: 2.5e+00/s
Starting n = 10
2023-10-02 12:17:48,170 subsystems 0/1 INFO :: Building subproblem matrices 1/1 (~100%) Elapsed: 0s, R

# Rational Approximation

In [12]:
rational_results = []

In [13]:
for n in n_vals:
    print(f"Starting n = {n}")

    rational_approximator = CauchySimplex(n, n, hot_start=True, max_iter=500).fit(x, y)
    
    solver = solve_multiple_coefficient_bessel(rational_approximator, a, m, Lx=0, Ux=np.log(2)/a, 
                                               Nx=256, dtype=np.float64, n_eigenvals=20)
    
    evals = np.sort(solver.eigenvalues.real)

    ratio_dataframe = pd.DataFrame(abs(np.sqrt(evals) - true_eigenvalues), columns=['Eigenvalue Errors'])
    ratio_dataframe['Approximator'] = 'Rational'
    ratio_dataframe['Num. Coefs'] = n
    
    rational_results.append(ratio_dataframe)

Starting n = 4
2023-10-02 12:17:50,879 subsystems 0/1 INFO :: Building subproblem matrices 1/1 (~100%) Elapsed: 0s, Remaining: 0s, Rate: 4.8e+00/s
Starting n = 5
2023-10-02 12:17:51,166 subsystems 0/1 INFO :: Building subproblem matrices 1/1 (~100%) Elapsed: 0s, Remaining: 0s, Rate: 4.1e+00/s
Starting n = 6
2023-10-02 12:17:51,569 subsystems 0/1 INFO :: Building subproblem matrices 1/1 (~100%) Elapsed: 0s, Remaining: 0s, Rate: 3.5e+00/s
Starting n = 7
2023-10-02 12:17:51,935 subsystems 0/1 INFO :: Building subproblem matrices 1/1 (~100%) Elapsed: 0s, Remaining: 0s, Rate: 3.1e+00/s
Starting n = 8
2023-10-02 12:17:52,342 subsystems 0/1 INFO :: Building subproblem matrices 1/1 (~100%) Elapsed: 0s, Remaining: 0s, Rate: 2.8e+00/s
Starting n = 9
2023-10-02 12:17:52,802 subsystems 0/1 INFO :: Building subproblem matrices 1/1 (~100%) Elapsed: 0s, Remaining: 0s, Rate: 2.5e+00/s
Starting n = 10
2023-10-02 12:17:53,300 subsystems 0/1 INFO :: Building subproblem matrices 1/1 (~100%) Elapsed: 0s, R

# Results

In [14]:
results_df = pd.concat(polynomial_results + rational_results)

In [15]:
results_df.head()

,Eigenvalue Errors,Approximator,Num. Coefs
0,0.000212,Polynomial,4
1,0.001484,Polynomial,4
2,0.002249,Polynomial,4
3,0.000598,Polynomial,4
4,0.003209,Polynomial,4


In [16]:
error_range = results_df.groupby(['Num. Coefs', 'Approximator']).max() \
                - results_df.groupby(['Num. Coefs', 'Approximator']).min()

error_range.columns = ['Max Error - Min Error']

In [17]:
average_error = results_df.groupby(['Num. Coefs', 'Approximator']).mean()

In [18]:
results = pd.concat([error_range, average_error], axis=1).reset_index()
results = results.pivot_table(index='Num. Coefs', columns='Approximator')

In [19]:
print(results.to_latex(float_format=lambda x: f"{x:.4e}"))

\begin{tabular}{lrrrr}
\toprule
 & \multicolumn{2}{r}{Eigenvalue Errors} & \multicolumn{2}{r}{Max Error - Min Error} \\
Approximator & Polynomial & Rational & Polynomial & Rational \\
Num. Coefs &  &  &  &  \\
\midrule
4 & 4.1888e-02 & 7.9795e-05 & 8.6690e-02 & 2.7114e-04 \\
5 & 2.2415e-03 & 4.2772e-06 & 5.7719e-03 & 1.6971e-05 \\
6 & 1.4512e-04 & 1.6216e-08 & 5.0030e-04 & 7.2248e-08 \\
7 & 4.0651e-06 & 1.7213e-11 & 1.6292e-05 & 4.4031e-11 \\
8 & 1.2205e-07 & 3.9424e-12 & 5.6306e-07 & 9.6909e-12 \\
9 & 2.2847e-09 & 1.3109e-12 & 9.5004e-09 & 4.4480e-12 \\
10 & 1.2554e-10 & 4.3703e-13 & 2.9574e-10 & 1.8181e-12 \\
11 & 2.6813e-11 & 3.0657e-10 & 7.7361e-11 & 1.1138e-09 \\
12 & 2.6194e-11 & 1.3935e-09 & 7.6032e-11 & 4.5828e-09 \\
13 & 2.6151e-11 & 3.0991e-10 & 7.5919e-11 & 1.2948e-09 \\
14 & 2.6209e-11 & 6.3271e-10 & 7.6112e-11 & 2.6343e-09 \\
15 & 2.5694e-11 & 5.6791e-10 & 7.4406e-11 & 1.7449e-09 \\
\bottomrule
\end{tabular}

